Exemplo abrindo TMAX com Google Earth Engine e plotando para uma posição a temperatura média no Colab

In [ ]:
# instalando geemap atualizado (https://geemap.org/)
%pip install -U "geemap[workshop]"

In [ ]:
import ee
import geemap
import numpy as np
import datetime
import plotly.express as px

In [2]:
# vai pedir a autentificação (tem que ter conta gmail)
m = geemap.Map()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=_yBEf_nkIrNAS0OWRQXGNKNTtctbcO7OKmlCfPTMAXc&tc=3prb0IpoFmImgB25tlw9dBJVi3P91kzLzhM8Z8F_y0Y&cc=W0fBHihMdpcXA29-vVhDxYIcbtevmbXkQ5oSRZhr8-4

The authorization workflow will generate a code, which you should paste in the box below.

Successfully saved authorization token.


In [ ]:
# pegando precipitação
var = ee.ImageCollection("projects/sat-io/open-datasets/BR-DWGD/PR")

# para reescalonar
offset = 225
scale  = 0.006866665

# reescalonando
def scaleBand(image):
    scaledImage = image.select("b1").rename(['pr']).multiply(scale).add(offset)
    return scaledImage.copyProperties(image, image.propertyNames())

var = var.map(scaleBand)
color_pal = ['000066', '001199', '0044BB', '0077DD', '33AAEE', '66CCFF', 'FFDDCC', 'FFBB99', 'FF9966', 'FF6644']
vis = {'min': 0,'max': 70, 'palette': color_pal}

# definindo periodo (tem limite)
tmax = var.filterDate('2010-01-01','2022-12-31').select(['TMAX']);
m.addLayer(var.first(), vis, 'PR')

In [ ]:
# plotando
m.centerObject(tmax.first())
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
# coordenada [lon, lat]
lon_lat = [-43.04, -14.33]
point = {'type':'Point', 'coordinates': lon_lat}
info = var.getRegion(point, 500).getInfo()

In [ ]:
header = info[0]
data = np.array(info[1:])
iTime = header.index('time')
time = [datetime.datetime.fromtimestamp(i/1000) for i in (data[0:,iTime].astype(int))]

# plotando
fig = px.scatter(x=time, y=data[:,-1].astype('float'),
                 labels={'x': 'Data', 'y': 'Precipitação (mm)'}, title=f"Precipitação na lat={lon_lat[1]} elon={lon_lat[0]}")
fig.show()